In [80]:
import pandas as pd
import numpy as np
import random

**Exercise 1**

In [79]:
def load_Xy(data):
    data = pd.read_csv(data)
    data_array = data.to_numpy()
    X,y = data_array[:,:-1], data_array[:,-1]
    y[y==0] = -1 
    return X,y

data = load_Xy('data.csv')


**Exercise 2**

In [84]:
def train_test_random(X,y)
    r = random.random()
    return X_train, X_test, y_train, y_test

0.35406624423881894


**Exercise 3**

**Exercise 4**

**Exercise 5**

**Exercise 6**